### Проверка того, что наше api работает

Перед началом необходимо собрать docker-образ и запустить контейнер

1. Следовать инструкциям https://github.com/fimochka-sudo/GB_docker_flask_example/blob/main/README.md
2. Далее уже запускать код ниже

In [1]:
import pandas as pd
from sklearn.metrics import roc_auc_score,roc_curve,scorer
from urllib import request, parse

X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

/home/ksn38/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [4]:
X_test[['Pclass', 'Sex', 'Age', 'Embarked']].head(3)

,Pclass,Sex,Age,Embarked
0,3,1,0.75,2.0
1,2,1,24.00,1.0
2,3,0,22.00,1.0


In [2]:
import urllib.request
import json      

def get_prediction(Pclass, Sex, Age, Embarked):
    body = {'Pclass': Pclass, 
                            'Sex': Sex,
                            'Age': Age,
           'Embarked': Embarked} 

    myurl = "http://127.0.0.1:8180/predict"
    req = urllib.request.Request(myurl)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    jsondata = json.dumps(body)
    jsondataasbytes = jsondata.encode('utf-8')   # needs to be bytes
    req.add_header('Content-Length', len(jsondataasbytes))
    print (jsondataasbytes)
    response = urllib.request.urlopen(req, jsondataasbytes)
    return json.loads(response.read())['predictions']

In [3]:
get_prediction(1, 0, 10, 2)

b'{"Pclass": 1, "Sex": 0, "Age": 10, "Embarked": 2}'


0.6563550687275211

In [7]:
%%time
predictions = X_test[['description', 'company_profile', 'benefits']].iloc[:500].apply(lambda x: get_prediction(x[0], 
                                                                                               x[1], 
                                                                                               x[2]), 1)

CPU times: user 340 ms, sys: 55.7 ms, total: 396 ms
Wall time: 5.16 s


In [8]:
roc_auc_score(y_score=predictions.values, y_true=y_test.iloc[:500])

0.9803161348046525

In [7]:
#X_test[['description', 'company_profile', 'benefits']].head(3)

Давайте подумаем, где наше api может сломаться

In [9]:
get_prediction("", "", None)

0.13518841848029955